In [1]:
from Functions import*
import matplotlib.pyplot as plt

# connect to the LCR meter
ser = DeviceConnect('COM5')

# define your frequencies
freq = Frequencies(100, 1e5, 3)

# set which measurements you'd like to use as its own element in a list
# mainReadings = ["R", "C", "L", "Z"]
# minorReadings = ["X", "D", "Q", "THR" (θ), "ESR"]
mainReadings = ['Z']
minorReadings = ['THR']

try:
    # ser - serial connection
    # freq - user frequencies (Hz)
    # mainReadings - inductance measurements (Ohm, H, C)
    # minorReadings - secondary inducatce measurement (deg, Ohm, None)
    # voltage - the voltage applied between electrodes (V)
    # biasVoltage - the biasvoltage (V)
    voltage = 1 
    biasVoltage = 0
    dataFrame = Experiment(ser, freq, mainReadings, minorReadings, voltage, biasVoltage)
    DeviceDisconnect(ser)
except: 
    print('Test Aborted')
    DeviceDisconnect(ser)     


[[ 9.47519e+07 -3.19652e+01]
 [ 9.47519e+07 -3.19652e+01]
 [ 9.39456e+07 -3.23420e+01]
 [ 9.47044e+07 -3.30908e+01]
 [ 9.44164e+07 -3.25176e+01]
 [ 9.53295e+07 -3.28810e+01]
 [ 9.43393e+07 -3.24979e+01]
 [ 9.53995e+07 -3.23889e+01]
 [ 9.45253e+07 -3.24223e+01]
 [ 9.44607e+07 -3.34659e+01]
 [ 9.43208e+07 -3.28738e+01]
 [ 9.43265e+07 -3.25436e+01]
 [ 9.36111e+07 -3.26619e+01]
 [ 9.45325e+07 -3.21655e+01]
 [ 9.40955e+07 -3.24759e+01]
 [ 9.45830e+07 -3.27616e+01]
 [ 9.41234e+07 -3.30030e+01]
 [ 9.50019e+07 -3.26008e+01]
 [ 9.47065e+07 -3.27980e+01]
 [ 9.43970e+07 -3.27087e+01]]
[[ 9.47781e+07 -3.26454e+01]
 [ 6.11064e+06 -8.57407e+01]
 [ 6.11872e+06 -8.57096e+01]
 [ 6.11519e+06 -8.57114e+01]
 [ 6.11657e+06 -8.57353e+01]
 [ 6.11488e+06 -8.57560e+01]
 [ 6.11298e+06 -8.57073e+01]
 [ 6.11534e+06 -8.57359e+01]
 [ 6.11619e+06 -8.57693e+01]
 [ 6.11596e+06 -8.57567e+01]
 [ 6.11196e+06 -8.57631e+01]
 [ 6.11118e+06 -8.57055e+01]
 [ 6.11348e+06 -8.57435e+01]
 [ 6.11266e+06 -8.57229e+01]
 [ 6.11483e+0